In [10]:
LR_EXPERIMENT_FILE = "lrjob-heartdisease.yaml"

In [11]:
import re

from IPython.utils.capture import CapturedIO


def get_resource(captured_io: CapturedIO) -> str:
    """
    Gets a resource name from `kubectl apply -f <configuration.yaml>`.

    :param str captured_io: Output captured by using `%%capture` cell magic
    :return: Name of the Kubernetes resource
    :rtype: str
    :raises Exception: if the resource could not be created
    """
    out = captured_io.stdout
    matches = re.search(r"^(.+)\s+created", out)
    if matches is not None:
        return matches.group(1)
    else:
        raise Exception(f"Cannot get resource as its creation failed: {out}. It may already exist.")

In [12]:
%%writefile $LR_EXPERIMENT_FILE
apiVersion: kubeflow.org/v1alpha3
kind: Experiment
metadata:
  name: heart
  namespace: sooter
spec:
  parallelTrialCount: 3
  maxTrialCount: 12
  maxFailedTrialCount: 3
  objective:
    type: maximize
    goal: 0.99
    objectiveMetricName: accuracy
  algorithm:
    algorithmName: bayesianoptimization
  metricsCollectorSpec:
    kind: StdOut
  parameters:
    - name: max_iter
      parameterType: int
      feasibleSpace:
        min: "100"
        max: "500"
    - name: penalty
      parameterType: categorical
      feasibleSpace:
        list: 
            - "l1"
            - "l2"
            - "elasticnet"
            - "none"
    - name: solver
      parameterType: categorical
      feasibleSpace:
        list:
            - "liblinear"
            - "newton-cg"
            - "lbfgs"
            - "sag"
            - "saga"
  trialTemplate:
    primaryContainerName: training-container
    trialParameters:
      - name: maxiter
        description: The number of iterations for solvers to converge
        reference: max_iter
      - name: penalty
        description: The norm used in penalization
        reference: penalty
      - name: solver
        description: Algorithm for optimization
        reference: solver
    trialSpec:
      apiVersion: batch/v1
      kind: Job
      spec:
        template:
          metadata:
            annotations:
              sidecar.istio.io/inject: "false"
          spec:
            containers:
              - name: training-container
                image: "docker.io/mavencodevv/lrjobheart:v.0.1"
                command:
                  - python3
                  - -u
                  - /lrjobheart.py
                args:
                  - "--max_iter=${trialParameters.max_iter}"
                  - "--penalty=${trialParameters.penalty}"
                  - "--solver=${trialParameters.solver}"
            restartPolicy: Never

Overwriting lrjob-heartdisease.yaml
